### Uyen Pham
### 508-Final Project
### Preliminary processing

#### +++++++++++++++++++++++++++++++++++++++++++++

## Columns of interested 
#### year: year compensation is paid
#### department: employee's department
#### job_title
#### base_salary
#### overtime: overtime pay
#### irregular_cash: vaction, mics pay
#### total_cash: sum of base_salary, overtime, and irregular_cash
#### retirement: retirement contribution by employer
#### health: healthcare, vision, dental
#### other_benefits: long term disability, medicare (LA has none)
#### total_benefits: sum of retirement, health, and other_benefits
#### total_compensation: sum of total_cash and total_benefits
#### city_id: San Jose: 1, San Francisco: 2, and Los Angles: 3

#### +++++++++++++++++++++++++++++++++++++++++++++++++++

In [40]:
import numpy as np
import pandas as pd
import sqlite3 as sq
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [41]:
#load San Jose dataset and add column year in each dataset accordingly
df1 = pd.read_csv('sj_compensation_2013.csv')
df1['year'] = 2013
df2 = pd.read_csv('sj_compensation_2014.csv')
df2['year'] = 2014
df3 = pd.read_csv('sj_compensation_2015.csv')
df3['year'] = 2015
df4 = pd.read_csv('sj_compensation_2016.csv')
df4['year'] = 2016
df5 = pd.read_csv('sj_compensation_2017.csv')
df5['year'] = 2017
##the file had some encoding error with utf-8 encoding error, so encoding was changed
df6 = pd.read_csv('sj_compensation_2018.csv', encoding='ISO-8859-1') 
df6['year'] = 2018
df7 = pd.read_csv('sj_compensation_2019.csv', encoding='ISO-8859-1')
df7['year'] = 2019
df8 = pd.read_csv('sj_compensation_2020.csv', encoding='ISO-8859-1')
df8['year'] = 2020
df9 = pd.read_csv('sj_compensation_2021.csv', encoding='ISO-8859-1')
df9['year'] = 2021

#Load San Francisco dataset
df_sf= pd.read_csv('sf_compensation.csv')

#Load Los Angeles dataset
df_la= pd.read_csv('la_compensation.csv')

In [42]:
#Re-format column names
dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df_sf, df_la]

for d in dfs:
    #Replace space with underscore in column names
    d.columns = [j.replace(' ','_') for j in d.columns] 
     #Replace colume name with capitalized letter to lower case
    d.columns = d.columns.str.replace(r'(\w+)', lambda x: x.group().lower(),
                                    n=2, regex=True)
   

In [43]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7821 entries, 0 to 7820
Data columns (total 14 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   name                                                 7821 non-null   object 
 1   department                                           7821 non-null   object 
 2   job_title_(as_of_12/31/15)                           7821 non-null   object 
 3   total_cash_compensation                              7821 non-null   object 
 4   base_pay                                             7716 non-null   object 
 5   overtime                                             4262 non-null   object 
 6   sick_and_vacation_payouts                            569 non-null    object 
 7   other_cash_compensation                              5605 non-null   object 
 8   defined_contribution_plan_contributions_-_city_paid  2527 non-null  

## San Jose

### Rename columns for consistency within the SJ datasets

In [44]:
#Rename columns
df1 = df1.rename(columns= {'department_name':"department", 
                           "deferred_comp_paid_by_employer":"defined_contribution_plan_contributions_city_paid", 
                           "employer's_share_pension_payments": "retirement_contribution",
                          'medical_dental_vision_city_paid':'medical_dental_vision'})

df2 = df2.rename(columns= {'department_name':"department", 
                           'job_title_(as_of_12/31/14)':"job_title", 
                           "deferred_comp_paid_by_employer":"defined_contribution_plan_contributions_city_paid",
                          "employer's_share_pension_payments": "retirement_contribution",
                          'medical_dental_vision_city_paid':'medical_dental_vision'})

df3 = df3.rename(columns= {'job_title_(as_of_12/31/15)':"job_title",
                           'defined_contribution_plan_contributions_-_city_paid':'defined_contribution_plan_contributions_city_paid',
                           'retirement_contributions_-_city_paid*':"retirement_contribution"})
df4 = df4.rename(columns= {'job_title_(as_of_12/31/16)':"job_title",
                           'defined_contribution_plan_contributions_-_city_paid':'defined_contribution_plan_contributions_city_paid',
                           'retirement_contributions_-_city_paid*':"retirement_contribution"})
df5 = df5.rename(columns= {'job_title_(as_of_12/31/17)':"job_title",
                           'defined_contribution_plan_contributions_-_city_paid':'defined_contribution_plan_contributions_city_paid',
                           'retirement_contributions_(normal_cost)_-_City_Paid*':"retirement_contribution"})
df6 = df6.rename(columns= {'job_title_(as_of_12/31/18)':"job_title",
                           'defined_contribution_plan_contributions_-_city_paid':'defined_contribution_plan_contributions_city_paid',
                           'retirement_contributions_(normal_cost)_-_City_Paid*':"retirement_contribution"})
df7 = df7.rename(columns= {'job_title_(as_of_12/31/19)':"job_title",
                           'defined_contribution_plan_contributions_-_city_paid':'defined_contribution_plan_contributions_city_paid',
                           'retirement_contributions_(normal_cost)_-_City_Paid':"retirement_contribution"})
df8 = df8.rename(columns= {'job_title_(as_of_12/31/20)':"job_title",
                           'defined_contribution_plan_contributions_-_city_paid':'defined_contribution_plan_contributions_city_paid',
                           'retirement_contributions_(normal_cost)_-_City_Paid':"retirement_contribution"})
df9 = df9.rename(columns= {'job_title_(as_of_12/31/21)':"job_title", 
                           'defined_contribution_plan_contributions_-_city_paid':'defined_contribution_plan_contributions_city_paid',
                           'retirement_contributions_(normal_cost)_-_City_Paid':"retirement_contribution"})


In [45]:
#combine San Jose datasets
combine_sj = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9], axis=0)
combine_sj.head()

,name,department,job_title,total_cash_compensation,base_pay,overtime,sick_and_vacation_payouts,other_cash_compensation,defined_contribution_plan_contributions_city_paid,medical_dental_vision,retirement_contribution,retired?,year,"long_term_disability,_life,_Medicare",misc_employment_related_costs
0,"Bustillos,Steven D",Police,Police Sergeant,286137.69,90888.0,89867.88,"97,969.79","7,412.02",NaN,"13,640.50","74,429.71",Yes,2013,NaN,NaN
1,"Figone,Debra J",City Manager,City Manager U,248564.84,227975.02,NaN,NaN,"20,589.82","20,517.64","15,166.00","131,032.44",Yes,2013,NaN,NaN
2,"Guerra,Daniel P",Police,Police Officer,241039.12,97198.4,132104.54,NaN,"11,736.18",NaN,"15,371.56","79,821.51",No,2013,NaN,NaN
3,"Moore,Christopher M",Police,Chief Of Police U,233540.32,15319.54,NaN,"215,470.80","2,749.98",NaN,"1,371.81","11,204.16",Yes,2013,NaN,NaN
4,"Vasquez,Hector M",Police,Police Officer,230469.85,97198.4,124552.69,NaN,"8,718.76",NaN,"16,345.32","79,821.51",No,2013,NaN,NaN


### Rename the columns for concistency with other city dataset

In [46]:
#rename columns
combine_sj = combine_sj.rename(columns= {'total_cash_compensation': 'total_cash',
                                   'base_pay':'base_salary',
                                   'medical_dental_vision':'health',
                                   'retired?':"retired",
                                   'long_term_disability,_life,_Medicare':'long_term_disability_life_medicare'})
                                   
    

In [47]:
combine_sj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71946 entries, 0 to 8161
Data columns (total 15 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   name                                               71946 non-null  object
 1   department                                         71946 non-null  object
 2   job_title                                          71946 non-null  object
 3   total_cash                                         71946 non-null  object
 4   base_salary                                        71247 non-null  object
 5   overtime                                           38998 non-null  object
 6   sick_and_vacation_payouts                          6941 non-null   object
 7   other_cash_compensation                            52655 non-null  object
 8   defined_contribution_plan_contributions_city_paid  22206 non-null  object
 9   health            

### change data type

In [48]:
#List of columns need to change dtype
column_list = ['total_cash', 'base_salary', 'overtime', 'sick_and_vacation_payouts', 
               'other_cash_compensation', 'defined_contribution_plan_contributions_city_paid', 
               'health', 'retirement_contribution', 
               'long_term_disability_life_medicare', 'misc_employment_related_costs']
#set function to eliminate commas in value and change dtype to float 
to_float = lambda x: float(x.replace(',', '')) if isinstance(x, str) and x else x

#Convert values 
combine_sj[column_list] = combine_sj[column_list].applymap(to_float)

### Generate additional columns

In [49]:
combine_sj['irregular_cash'] = combine_sj[['sick_and_vacation_payouts', 'other_cash_compensation']].sum(axis=1)

combine_sj['retirement'] = combine_sj[['defined_contribution_plan_contributions_city_paid', 
                                 'retirement_contribution']].sum(axis=1)

combine_sj['other_benefits'] = combine_sj[['long_term_disability_life_medicare', 
                                     'misc_employment_related_costs']].sum(axis=1)
combine_sj['total_benefits'] =  combine_sj[['retirement', 'health', 'other_benefits']].sum(axis=1)

combine_sj['total_compensation'] = combine_sj[['total_cash', 'total_benefits']].sum(axis=1)

combine_sj['city_id'] = 1

In [50]:
combine_sj.head()

,name,department,job_title,total_cash,base_salary,overtime,sick_and_vacation_payouts,other_cash_compensation,defined_contribution_plan_contributions_city_paid,health,...,retired,year,long_term_disability_life_medicare,misc_employment_related_costs,irregular_cash,retirement,other_benefits,total_benefits,total_compensation,city_id
0,"Bustillos,Steven D",Police,Police Sergeant,286137.69,90888.00,89867.88,97969.79,7412.02,NaN,13640.50,...,Yes,2013,NaN,NaN,105381.81,74429.71,0.0,88070.21,374207.90,1
1,"Figone,Debra J",City Manager,City Manager U,248564.84,227975.02,NaN,NaN,20589.82,20517.64,15166.00,...,Yes,2013,NaN,NaN,20589.82,151550.08,0.0,166716.08,415280.92,1
2,"Guerra,Daniel P",Police,Police Officer,241039.12,97198.40,132104.54,NaN,11736.18,NaN,15371.56,...,No,2013,NaN,NaN,11736.18,79821.51,0.0,95193.07,336232.19,1
3,"Moore,Christopher M",Police,Chief Of Police U,233540.32,15319.54,NaN,215470.80,2749.98,NaN,1371.81,...,Yes,2013,NaN,NaN,218220.78,11204.16,0.0,12575.97,246116.29,1
4,"Vasquez,Hector M",Police,Police Officer,230469.85,97198.40,124552.69,NaN,8718.76,NaN,16345.32,...,No,2013,NaN,NaN,8718.76,79821.51,0.0,96166.83,326636.68,1


In [51]:
combine_sj.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 71946 entries, 0 to 8161
Data columns (total 21 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   name                                               71946 non-null  object 
 1   department                                         71946 non-null  object 
 2   job_title                                          71946 non-null  object 
 3   total_cash                                         71946 non-null  float64
 4   base_salary                                        71247 non-null  float64
 5   overtime                                           38998 non-null  float64
 6   sick_and_vacation_payouts                          6941 non-null   float64
 7   other_cash_compensation                            52655 non-null  float64
 8   defined_contribution_plan_contributions_city_paid  22206 non-null  float64
 9   health 

In [52]:
#write the combined file to CSV file 
combine_sj.to_csv('sj_compens_v1.csv', index=False)

In [53]:
sj_v1= pd.read_csv('sj_compens_v1.csv')
#sj_v1.info()


/Users/lamnguyen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [54]:
#filtered_df = combine[combine['total_benefit'] > 0]
#filtered_df

In [55]:
col_names_sj = sj_v1.columns.tolist()
col_names_sj

['name',
 'department',
 'job_title',
 'total_cash',
 'base_salary',
 'overtime',
 'sick_and_vacation_payouts',
 'other_cash_compensation',
 'defined_contribution_plan_contributions_city_paid',
 'health',
 'retirement_contribution',
 'retired',
 'year',
 'long_term_disability_life_medicare',
 'misc_employment_related_costs',
 'irregular_cash',
 'retirement',
 'other_benefits',
 'total_benefits',
 'total_compensation',
 'city_id']

## San Francisco

In [58]:
#add city_id
df_sf['city_id'] = 2

In [59]:
df_sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799562 entries, 0 to 799561
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   organization_group_code  799562 non-null  int64  
 1   job_family_code          799562 non-null  object 
 2   job_code                 799562 non-null  object 
 3   year_type                799562 non-null  object 
 4   year                     799562 non-null  int64  
 5   organization_group       799562 non-null  object 
 6   department_code          799560 non-null  object 
 7   department               799560 non-null  object 
 8   union_code               799383 non-null  float64
 9   union                    799383 non-null  object 
 10  job_family               799562 non-null  object 
 11  job                      799557 non-null  object 
 12  employee_identifier      799562 non-null  int64  
 13  salaries                 799562 non-null  float64
 14  over

### Rename the columns for concistency with other city dataset

In [60]:
#rename columns
df_sf = df_sf.rename(columns= {'job':'job_title', 
                               'total_salary': 'total_cash',
                                'salaries':'base_salary',
                               'other_salaries':'irregular_cash',
                               'health_and_dental':'health'})
                                   

In [61]:
df_sf.head()

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,base_salary,overtime,irregular_cash,total_cash,retirement,health,other_benefits,total_benefits,total_compensation,city_id
0,3,1400,1404,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,790.0,"SEIU, Local 1021, Misc",...,60720.01,0.00,0.00,60720.01,13653.20,14733.76,4904.34,33291.30,94011.31,2
1,3,9700,9703,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,91677.00,0.00,0.00,91677.00,17524.20,14733.76,7411.13,39669.09,131346.09,2
2,3,2900,2918,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,89106.03,0.00,1540.00,90646.03,17327.20,14733.76,7401.92,39462.88,130108.91,2
3,3,2900,2918,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,85581.11,3355.94,337.75,89274.80,16359.16,14151.56,7096.21,37606.93,126881.73,2
4,3,2900,2905,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,86457.00,0.00,2090.00,88547.00,16925.97,14733.76,7257.89,38917.62,127464.62,2


In [62]:
col_names_sf = df_sf.columns.tolist()
col_names_sf

['organization_group_code',
 'job_family_code',
 'job_code',
 'year_type',
 'year',
 'organization_group',
 'department_code',
 'department',
 'union_code',
 'union',
 'job_family',
 'job_title',
 'employee_identifier',
 'base_salary',
 'overtime',
 'irregular_cash',
 'total_cash',
 'retirement',
 'health',
 'other_benefits',
 'total_benefits',
 'total_compensation',
 'city_id']

In [63]:
#write the combined file to CSV file 
df_sf.to_csv('sf_compens_v1.csv', index=False)

## Los Angeles

In [64]:
#Add city_id column
df_la['city_id'] = 3

In [65]:
df_la.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753959 entries, 0 to 753958
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   record_nbr                     753959 non-null  object 
 1   pay_year                       753959 non-null  int64  
 2   department_no                  753959 non-null  int64  
 3   department_title               753959 non-null  object 
 4   job_class_pgrade               753415 non-null  object 
 5   job_title                      753415 non-null  object 
 6   employment_type                753959 non-null  object 
 7   job_status                     753959 non-null  object 
 8   mou                            753273 non-null  object 
 9   mou_title                      753166 non-null  object 
 10  regular_pay                    753959 non-null  float64
 11  overtime_pay                   753525 non-null  float64
 12  all_other_pay                 

In [66]:
#convert dtype
column_list2 = ['regular_pay', 'overtime_pay', 'all_other_pay', 'total_pay', 
                'city_retirement_contributions', 'benefit_pay']
#function to eliminate commas and dollar-sign to float 
to_float = lambda x: float(x.replace(',', '').replace('$', '')) if isinstance(x, str) and x else x
#Convert values with comma to scientific one
df_la[column_list2] = df_la[column_list2].applymap(to_float)

### Rename the columns for concistency with other city dataset

In [73]:
#rename coumns
df_la = df_la.rename(columns= {'department_title':'department',
                               'pay_year':"year",
                               'regular_pay':'base_salary',
                               'overtime_pay':'overtime',
                               'all_other_pay':'irregular_cash',
                               'total_pay':'total_cash',
                               'city_retirement_contributions':'retirement',
                               'benefit_pay':'health'})

In [74]:
df_la.head()

,record_nbr,year,department_no,department,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,...,overtime,irregular_cash,total_cash,retirement,health,gender,ethnicity,city_id,total_benefits,total_compensation
0,303030303632,2017,98,WATER AND POWER,3156-5,CUSTODIAN,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,...,4785.05,2021.84,62532.13,3678.0,23508.9,FEMALE,HISPANIC,3,27186.9,89719.03
1,3030303036,2017,98,WATER AND POWER,9105-5,UTILITY ADMINISTRATOR,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,...,16340.50,6170.49,161685.87,9186.0,23508.9,FEMALE,ASIAN AMERICAN,3,32694.9,194380.77
2,303030313232,2017,98,WATER AND POWER,9602-4,WATER SERVICES MANAGER,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,...,0.00,12504.30,258383.42,16228.0,23508.9,MALE,BLACK,3,39736.9,298120.32
3,303030313632,2017,98,WATER AND POWER,5885-5,WTR TRTMT OPR,FULL_TIME,ACTIVE,6,STEAM PLANT AND WATER SUPPLY UNIT,...,7824.99,12630.52,121949.85,6699.0,23508.9,MALE,ASIAN AMERICAN,3,30207.9,152157.75
4,303030323632,2017,98,WATER AND POWER,3841-5,ELTL MCHC,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,...,22284.37,1566.75,125196.24,6689.0,23508.9,MALE,HISPANIC,3,30197.9,155394.14


### Generate additional columns

In [97]:
#create total_benefits colum
df_la['total_benefits'] =  df_la[['retirement', 'health']].sum(axis=1)
df_la['total_compensation'] = df_la [['total_cash', 'total_benefits']].sum(axis=1)
df_la['other_benefits'] = 'NaN'

In [98]:
#df_la.head()

In [99]:
col_names_la = df_la.columns.tolist()
col_names_la

['record_nbr',
 'year',
 'department_no',
 'department',
 'job_class_pgrade',
 'job_title',
 'employment_type',
 'job_status',
 'mou',
 'mou_title',
 'base_salary',
 'overtime',
 'irregular_cash',
 'total_cash',
 'retirement',
 'health',
 'gender',
 'ethnicity',
 'city_id',
 'total_benefits',
 'total_compensation',
 'other_benefits']

In [100]:
#write the combined file to CSV file 
df_la.to_csv('la_compens_v1.csv', index=False)

# Create Database and tables

In [101]:
connection = sq.connect('comps')
conn = connection.cursor()

## sj_table

In [80]:
#Read SJ data
table1 = pd.read_csv('sj_compens_v1.csv')
col_names_sj = table1.columns.tolist()
col_names_sj

/Users/lamnguyen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['name',
 'department',
 'job_title',
 'total_cash',
 'base_salary',
 'overtime',
 'sick_and_vacation_payouts',
 'other_cash_compensation',
 'defined_contribution_plan_contributions_city_paid',
 'health',
 'retirement_contribution',
 'retired',
 'year',
 'long_term_disability_life_medicare',
 'misc_employment_related_costs',
 'irregular_cash',
 'retirement',
 'other_benefits',
 'total_benefits',
 'total_compensation',
 'city_id']

In [81]:
#Create SJ table
conn.execute("CREATE TABLE IF NOT EXISTS sj_table" + 
             "(name string,\
             department string, \
             job_title string, \
             total_cash float, \
             base_salary float, \
             overtime float,\
             sick_and_vacation_payouts float,\
             other_cash_compensation float,\
             defined_contribution_plan_contributions_city_paid float,\
             health float,\
             retirement_contribution float,\
             retired string,\
             year int,\
             long_term_disability_life_medicare float,\
             misc_employment_related_costs float,\
             irregular_cash float, \
             retirement float,\
             other_benefits float, \
             total_benefits float, \
             total_compensation float, \
             city_id int)")

In [82]:
#save content from data frame to database table
table1.to_sql('sj_table', connection, if_exists='replace', index=False)

In [83]:
pd.read_sql(f'SELECT * FROM sj_table limit 5', connection)

,name,department,job_title,total_cash,base_salary,overtime,sick_and_vacation_payouts,other_cash_compensation,defined_contribution_plan_contributions_city_paid,health,...,retired,year,long_term_disability_life_medicare,misc_employment_related_costs,irregular_cash,retirement,other_benefits,total_benefits,total_compensation,city_id
0,"Bustillos,Steven D",Police,Police Sergeant,286137.69,90888.00,89867.88,97969.79,7412.02,NaN,13640.50,...,Yes,2013,None,None,105381.81,74429.71,0.0,88070.21,374207.90,1
1,"Figone,Debra J",City Manager,City Manager U,248564.84,227975.02,NaN,NaN,20589.82,20517.64,15166.00,...,Yes,2013,None,None,20589.82,151550.08,0.0,166716.08,415280.92,1
2,"Guerra,Daniel P",Police,Police Officer,241039.12,97198.40,132104.54,NaN,11736.18,NaN,15371.56,...,No,2013,None,None,11736.18,79821.51,0.0,95193.07,336232.19,1
3,"Moore,Christopher M",Police,Chief Of Police U,233540.32,15319.54,NaN,215470.80,2749.98,NaN,1371.81,...,Yes,2013,None,None,218220.78,11204.16,0.0,12575.97,246116.29,1
4,"Vasquez,Hector M",Police,Police Officer,230469.85,97198.40,124552.69,NaN,8718.76,NaN,16345.32,...,No,2013,None,None,8718.76,79821.51,0.0,96166.83,326636.68,1


## sf_table

In [84]:
#Read SF data
table2 = pd.read_csv('sf_compens_v1.csv')
col_names_sf = table2.columns.tolist()
col_names_sf

['organization_group_code',
 'job_family_code',
 'job_code',
 'year_type',
 'year',
 'organization_group',
 'department_code',
 'department',
 'union_code',
 'union',
 'job_family',
 'job_title',
 'employee_identifier',
 'base_salary',
 'overtime',
 'irregular_cash',
 'total_cash',
 'retirement',
 'health',
 'other_benefits',
 'total_benefits',
 'total_compensation',
 'city_id']

In [85]:
#Create table
conn.execute("CREATE TABLE IF NOT EXISTS sf_table" + 
             "('organization_group_code', \
             'job_family_code', \
             'job_code', \
             'year_type', \
             'year', \
             'organization_group', \
             'department_code', \
             'department', \
             'union_code', \
             'union', \
             'job_family', \
             'job_title', \
             'employee_identifier', \
             'base_salary', \
             'overtime', \
             'irregular_cash', \
             'total_cash', \
             'retirement', \
             'health', \
             'other_benefits', \
             'total_benefits', \
             'total_compensation', \
             city_id int)")

In [86]:
#save content from data frame to database table
table2.to_sql('sf_table', connection, if_exists='replace', index=False)
pd.read_sql(f'SELECT * FROM sf_table limit 5', connection)

,organization_group_code,job_family_code,job_code,year_type,year,organization_group,department_code,department,union_code,union,...,base_salary,overtime,irregular_cash,total_cash,retirement,health,other_benefits,total_benefits,total_compensation,city_id
0,3,1400,1404,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,790.0,"SEIU, Local 1021, Misc",...,60720.01,0.00,0.00,60720.01,13653.20,14733.76,4904.34,33291.30,94011.31,2
1,3,9700,9703,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,91677.00,0.00,0.00,91677.00,17524.20,14733.76,7411.13,39669.09,131346.09,2
2,3,2900,2918,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,89106.03,0.00,1540.00,90646.03,17327.20,14733.76,7401.92,39462.88,130108.91,2
3,3,2900,2918,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,85581.11,3355.94,337.75,89274.80,16359.16,14151.56,7096.21,37606.93,126881.73,2
4,3,2900,2905,Fiscal,2019,Human Welfare & Neighborhood Development,HSA,Human Services,535.0,"SEIU, Local 1021, Misc",...,86457.00,0.00,2090.00,88547.00,16925.97,14733.76,7257.89,38917.62,127464.62,2


## la_table

In [102]:
#Read LA data
table3 = pd.read_csv('la_compens_v1.csv')
col_names_la = table3.columns.tolist()
col_names_la

/Users/lamnguyen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['record_nbr',
 'year',
 'department_no',
 'department',
 'job_class_pgrade',
 'job_title',
 'employment_type',
 'job_status',
 'mou',
 'mou_title',
 'base_salary',
 'overtime',
 'irregular_cash',
 'total_cash',
 'retirement',
 'health',
 'gender',
 'ethnicity',
 'city_id',
 'total_benefits',
 'total_compensation',
 'other_benefits']

In [108]:
#Create table
conn.execute("CREATE TABLE IF NOT EXISTS la_table" + 
             "('record_nbr', \
             'year', \
             'department_no', \
             'department', \
             'job_class_pgrade', \
             'job_title', \
             'employment_type', \
             'job_status', \
             'mou', \
             'mou_title', \
             'base_salary', \
             'overtime', \
             'irregular_cash', \
             'total_cash', \
             'retirement', \
             'health', \
             'gender', \
             'ethnicity',\
             'city_id int', \
             'total_benefits'\
             'total_compensation', \
             other_benefits)")

In [109]:
#save content from data frame to database table
table3.to_sql('la_table', connection, if_exists='replace', index=False)
pd.read_sql(f'SELECT * FROM la_table limit 5', connection)

,record_nbr,year,department_no,department,job_class_pgrade,job_title,employment_type,job_status,mou,mou_title,...,irregular_cash,total_cash,retirement,health,gender,ethnicity,city_id,total_benefits,total_compensation,other_benefits
0,303030303632,2017,98,WATER AND POWER,3156-5,CUSTODIAN,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,...,2021.84,62532.13,3678.0,23508.9,FEMALE,HISPANIC,3,27186.9,89719.03,None
1,3030303036,2017,98,WATER AND POWER,9105-5,UTILITY ADMINISTRATOR,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,...,6170.49,161685.87,9186.0,23508.9,FEMALE,ASIAN AMERICAN,3,32694.9,194380.77,None
2,303030313232,2017,98,WATER AND POWER,9602-4,WATER SERVICES MANAGER,FULL_TIME,ACTIVE,M,MANAGEMENT EMPLOYEES UNIT,...,12504.30,258383.42,16228.0,23508.9,MALE,BLACK,3,39736.9,298120.32,None
3,303030313632,2017,98,WATER AND POWER,5885-5,WTR TRTMT OPR,FULL_TIME,ACTIVE,6,STEAM PLANT AND WATER SUPPLY UNIT,...,12630.52,121949.85,6699.0,23508.9,MALE,ASIAN AMERICAN,3,30207.9,152157.75,None
4,303030323632,2017,98,WATER AND POWER,3841-5,ELTL MCHC,FULL_TIME,ACTIVE,8,OPERATING MAINTENANCE AND SERVICE UNIT,...,1566.75,125196.24,6689.0,23508.9,MALE,HISPANIC,3,30197.9,155394.14,None


## Observe the differences in department between datasets

In [269]:
#pd.read_sql(f'SELECT DISTINCT department FROM la_table;', connection)

In [ ]:
#pd.read_sql(f'SELECT DISTINCT department FROM sf_table;', connection)

In [270]:
#pd.read_sql(f'SELECT DISTINCT department FROM sj_table;', connection)

# Merge 3 tables by rows

In [111]:
# The 3 table were merge based on the columns of interest
df = pd.read_sql(f'SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
total_cash, retirement, health, other_benefits, total_benefits, total_compensation, city_id \
FROM sj_table \
UNION ALL SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
total_cash, retirement, health, other_benefits, total_benefits, total_compensation, city_id \
FROM sf_table \
UNION ALL SELECT year, department, job_title, base_salary, overtime, irregular_cash, \
total_cash, retirement, health, other_benefits, total_benefits, total_compensation, city_id \
FROM la_table', connection)

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1625467 entries, 0 to 1625466
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   year                1625467 non-null  int64  
 1   department          1625465 non-null  object 
 2   job_title           1624918 non-null  object 
 3   base_salary         1624768 non-null  float64
 4   overtime            1592085 non-null  float64
 5   irregular_cash      1625033 non-null  float64
 6   total_cash          1625467 non-null  float64
 7   retirement          1625467 non-null  float64
 8   health              1606480 non-null  float64
 9   other_benefits      871508 non-null   float64
 10  total_benefits      1625467 non-null  float64
 11  total_compensation  1625467 non-null  float64
 12  city_id             1625467 non-null  int64  
dtypes: float64(9), int64(2), object(2)
memory usage: 161.2+ MB
